<a href="https://colab.research.google.com/github/horizon245/Cat_or_dog/blob/main/Garbage_Sorter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.3/235.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Garbage\ Porject/garbage_classification.zip -d /content/dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/Garbage classification/Garbage classification/cardboard/cardboard152.jpg  
  inflating: /content/dataset/Garbage classification/Garbage classification/cardboard/cardboard153.jpg  
  inflating: /content/dataset/Garbage classification/Garbage classification/cardboard/cardboard154.jpg  
  inflating: /content/dataset/Garbage classification/Garbage classification/cardboard/cardboard155.jpg  
  inflating: /content/dataset/Garbage classification/Garbage classification/cardboard/cardboard156.jpg  
  inflating: /content/dataset/Garbage classification/Garbage classification/cardboard/cardboard157.jpg  
  inflating: /content/dataset/Garbage classification/Garbage classification/cardboard/cardboard158.jpg  
  inflating: /content/dataset/Garbage classification/Garbage classification/cardboard/cardboard159.jpg  
  inflating: /content/dataset/Garbage classification/Garbage classification/cardboard/cardboard16.jpg  
  inf

In [ ]:
!pip install tensorflow opencv-python scikit-learn

In [ ]:
import os
print(os.listdir('/content/dataset'))

['zero-indexed-files.txt', 'one-indexed-files-notrash_test.txt', 'one-indexed-files-notrash_val.txt', 'one-indexed-files.txt', 'one-indexed-files-notrash_train.txt', 'Garbage classification', 'garbage classification']


In [ ]:
import os
print(os.listdir('/content'))

['.config', 'drive', 'dataset', 'sample_data']


In [ ]:
print(os.listdir('/content/dataset/Garbage classification'))  # Adjust the name based on the output above

['Garbage classification']


In [ ]:
import os
print(os.listdir('/content/dataset/Garbage classification/Garbage classification'))

['plastic', 'glass', 'metal', 'paper', 'cardboard', 'trash']


In [ ]:
import cv2
import os
import numpy as np

dataset_path = '/content/dataset/Garbage classification/Garbage classification'
images = []
labels = []

for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (224, 224))
                img = img / 255.0
                images.append(img)
                labels.append(folder)
            else:
                print(f"Failed to load image: {img_path}")

images = np.array(images)
labels = np.array(labels)
print(f"Loaded {len(images)} images with shape {images.shape}")
print(f"Labels: {np.unique(labels)}")

Loaded 2527 images with shape (2527, 224, 224, 3)
Labels: ['cardboard' 'glass' 'metal' 'paper' 'plastic' 'trash']


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 3,575,878 (13.64 MB)

 Trainable params: 1,317,894 (5.03 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape[0]} images")
print(f"Test set: {X_test.shape[0]} images")

Training set: 2021 images
Test set: 506 images


In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - accuracy: 0.5944 - loss: 1.1454 - val_accuracy: 0.8063 - val_loss: 0.4957
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - accuracy: 0.8650 - loss: 0.3672 - val_accuracy: 0.7905 - val_loss: 0.5511
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.9098 - loss: 0.2570 - val_accuracy: 0.8478 - val_loss: 0.4300
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.9755 - loss: 0.1055 - val_accuracy: 0.8379 - val_loss: 0.4509
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.9830 - loss: 0.0703 - val_accuracy: 0.8557 - val_loss: 0.4308
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.9998 - loss: 0.0266 - val_accuracy: 0.8340 - val_loss: 0.4993
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.9955 - loss: 0.0253 - val_accuracy: 0.8636 - val_loss: 0.4309
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.9987 - loss: 0.0128 - val_accuracy: 0.8636 - v

In [ ]:
from google.colab import files

# Upload the image
uploaded = files.upload()

# Get the filename of the uploaded image
image_filename = list(uploaded.keys())[0]  # Takes the first uploaded file
print(f"Uploaded image: {image_filename}")

Saving gettyimages-647762772-612x612.jpg to gettyimages-647762772-612x612 (1).jpg
Uploaded image: gettyimages-647762772-612x612 (1).jpg


In [ ]:
import cv2
import numpy as np

# Load the image
img = cv2.imread(image_filename)

# Preprocess the image
img = cv2.resize(img, (224, 224))  # Resize to 224x224
img = img / 255.0  # Normalize pixel values
img = np.expand_dims(img, axis=0)  # Add batch dimension (1, 224, 224, 3)

print(f"Preprocessed image shape: {img.shape}")

Preprocessed image shape: (1, 224, 224, 3)


In [ ]:
# Make a prediction
prediction = model.predict(img)

# Get the predicted class index
predicted_class_idx = np.argmax(prediction, axis=1)[0]

# Map the class index back to the category name
categories = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
predicted_category = categories[predicted_class_idx]

# Print the prediction and confidence
confidence = prediction[0][predicted_class_idx] * 100
print(f"Predicted category: {predicted_category}")
print(f"Confidence: {confidence:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Predicted category: cardboard
Confidence: 88.66%
